# Evaluate Input Script Parameters
* Set Parameters from input script
* Evaluate Elementary Process Rates
* Evaluate Deposition Parameters
* Evaluate Substrate Parameters
* more cool stuff

### import modules

In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import importlib
from pathlib import Path

In [ ]:
import KMC_Deposition_v3p4p1 as KMC_Dep
import KMC_Miscellaneous_v3p4p1 as KMC_Misc
import KMC_Analysis_v3p4p1 as KMC_Analysis
import KMC_Processes_v3p4p1 as KMC_Proc

In [ ]:
importlib.reload(KMC_Dep)
importlib.reload(KMC_Misc)

### Set Parameters

In [ ]:
Parameters = {}     # Create Parameters Dictionary

Parameters['Current_Version'] = 'v3.4.1'   # version of KMC running


# Simulation Parameters
Parameters['Project_Name'] = 'Density'      # Name for the project save directory.
Parameters['Simulation_Name'] = 'R88_Test'          # Name of particular simulation within a project. Can be an empty string, '', because it's DateTime stamped

Parameters['Enable_Print_Outs'] = True          # Enable/Disable simulation prinouts.
Parameters['Enable_Plots'] = True               # Display final plots at end of simulation. Set to False if running multiple simulations

Parameters['Solver_Type'] = 'Binary'            # Type of Solver: 'Linear' O(N^2), 'Binary' O(N log N), 'Set' O(N log N) (currently not wokring)

Parameters['Simulation_Type'] = 'Deposition'    # Simulation Type. Choices: 'Deposition', 'Diffusion', 'Density', 'Ising' (future)
Parameters['Dimension'] = '3D'                  # Either 2D (1+1) or 3D (2+1). For 2D only Lx is used.

Parameters['Number_of_Simulations'] = 1         # Number of back-to-back simulations to perform (fixed parameters). Analysis results will be averaged.



# Save Data Parameters - (Note: Data is now saved at each analysis time step)
Parameters['Save_All_Events'] = False          # Saves all events in the simulation to output file.
Parameters['Save_Lattice'] = False             # Save snapshot of lattice at specified times.
Parameters['Save_Surface'] = False             # Save a top down view of surface at specified times.
Parameters['Save_Time_Step'] = 0.1             # Time interval to save lattice and statistics data (set to resolution needed for post analysis). Also for Progress Bar update.
Parameters['Save_with_Analysis'] = False       # Save data at same time as analysis. This overrides the above time step (though it is still used for progress bar). 



# On-the-Fly Analysis Parameters - Note: currently only enabled for 3D Deposition simulations
Parameters['Coverage'] = True                  # do a coverage analysis
Parameters['Island_Analysis'] = False           # do island size distribution analysis
Parameters['Structure_Factor'] = False          # do structure factor analysis
Parameters['Gaussian_Filter'] = 1.0            # apply gaussian filter to SF analysis. 0 = no filter, > 0 = sigma value

Parameters['Analysis_Style'] = 'time'              # choose whether to analyze simulation by time step ('time') or simulation steps ('step')
Parameters['Analysis_Time_Spacing'] = 'linear'     # delta t spacing of analysis points. Options: 'linear', 'quad', 'log'. For 'log' this is time of first data point
Parameters['Log_Points'] = 100                     # For log spacing option only. This is the number of points per pulse. Set 'Analysis_Delta_Time' to be the first analysis point

Parameters['Analysis_Delta_Time'] = 0.1      # time interval for On-the-Fly analysis in seconds. Set 'Analysis_Style' to 'time'
Parameters['Analysis_Delta_Step'] = 1e4      # step interval for On-the-Fly analysis. This will result in non-linear time step for analysis points. Set 'Analysis_Style' to 'step'

Parameters['On_the_Fly_Save'] = False         # whether to save the on-the-fly results for each simulation. Usually true, unless doing multiple simulations.
Parameters['Average_Results'] = False          # average on-the-fly results and save across mulit-simulations. Only used when running multiple simulations.



# Substrate Parameters
Parameters['Lx'] = 200                   # Substrate size in x direction
Parameters['Ly'] = 200                   # Substrate size in y direction (not used in 2D mode)
Parameters['depth'] = 5                 # Substrate size in z direction. Give yourself enough layers. (be careful setting this, or errors could occur)

Parameters['Substrate_Type'] = 'Islands'           # Substrate Type. Options: 'Flat', 'Islands', 'Steps', 'Custom'

Parameters['Feature_Grid'] = 'Square'             # Feature Grid: 'Square', 'Hex': Hexagonal. This is for 'Islands' type only.
Parameters['Feature_Layout'] = 'Uniform'          # Feature Layout: 'Uniform': uniform grid, 'Correlated': random offset from uniform , 'Random': random layout
Parameters['Feature_Spacing'] = (8,8,0)         # (number in x direction, number in y, avg spread of features (sigma)). For 'Islands' only.

Parameters['Size_Distribution'] = 'Gaussian'        # Size ditribution of features: 'None', 'Gaussian', or 'Correlated'
Parameters['Size_Values'] = (11,1,0)                # (Avg Radius or Step Length, Height of features, Width of distribution)

Parameters['Substrate_Particle_State'] = 'Passive'      # Whether substrate particles are active or not. Options: 'Active', 'Passive'.

Parameters['Substrate_File'] = 'Surface_Cov0.6.npy'          # For custom substrate, name of coordinates file. File must be in same directory as simulation script.



# Deposition Parameters (Deposition type simulations only)
Parameters['Deposition_Type'] = 'PLD'              # Type of deposition simulation. Options: 'PLD' or 'MBE'
Parameters['Pulses_per_Layer'] = 20                # number of pulses per monolayer. For PLD only
Parameters['Layers'] = 0.05                        # number of monolayers to simulate

Parameters['Dwell_Time'] = 10.0                    # Total pulse period (for PLD) or total deposition time (for MBE) in seconds.
Parameters['Pulse_Shape'] = 'Uniform'              # Shape of the deposition pulse. Options: 'Uniform', 'Non_Uniform'
Parameters['Pulse_Width'] = 1e-8                   # Width of deposition pulse (for PLD) or total deposition time (for MBE, set the same as dwell time) in seconds.

Parameters['Post_Anneal'] = False                  # Run a Post Anneal Step (True/False)
Parameters['Post_Anneal_Time'] = 100.0             # Length of Post Anneal step in seconds.



# Density Map Model Parameters (Density map type simulations only)
Parameters['Total_Sims'] = 10                        # total number of sims at each time


# Diffusion Only Parameters (Diffusion type simulations only)
Parameters['Cluster'] = np.array([[0,1,2],[0,0,0]])                                                 # set up cluster coordinates for surface diffusion
Parameters['Cluster'] = np.array([[int(0.5*Parameters['Lx'])],[int(0.5*Parameters['Ly'])]])         # one particle in the middle

Parameters['Sim_Time'] = 1.0            # simulation time in seconds



# Thermal Processes and Activation Energies (Slow Dynamics)
Parameters['Enable_Processes'] = 'No_Uphill'      # Choose which processes to enable. Options: 'All', 'No_Uphill_4NN', 'No_Uphill', 'No_Detach', 'No_Edge'

Parameters['w0'] = 10**13                       # Prefactor for Arrhenius law (Crystal Vibration Frequency: w0 ~ kT/h )
Parameters['Substrate_Temperature'] = 600       # Substrate temperature in Celcius

Parameters['Ea_diffusion'] = 1.8                # Energy Barrier for surface diffusion in eV
Parameters['Ea_ehrlich_schwoebel'] = 0.0        # Energy Barrier for downhill/uphill diffusion in eV
Parameters['Ea_detach'] = 0.0                   # Energy Barrier for detachment from one nearest neighbor in eV
Parameters['Ea_edge'] = 0.0                     # Energy Barrier for Edge diffusion in eV (3D only). Value of 0.0 means same rate as surface diffusion
Parameters['Ea_corner'] = 10.0                   # Energy Barrier for Corner diffusion in eV (3D only)



# Non-Thermal Processes (Fast Dynamics)
Parameters['Downward_Funneling'] = False      # enables downward funneling when particles land on step edges
Parameters['Transient_Mobility'] = False      # enables transient mobility
Parameters['Island_Chipping'] = False         # enable island chipping

# Analysis Times
* Set up non-lienar analysis times

In [ ]:
# Reset relevant parmaters
Parameters['Analysis_Style'] = 'time'              # choose whether to analyze simulation by time step ('time') or simulation steps ('step')
Parameters['Analysis_Time_Spacing'] = 'quad'     # delta t spacing of analysis points. Options: 'linear', 'quad', 'log'. For 'log' this is time of first data point
Parameters['Log_Points'] = 200                     # For log spacing option only. This is the number of points per pulse. Set 'Analysis_Delta_Time' to be the first analysis point

Parameters['Analysis_Delta_Time'] = 1e-4     # time interval for On-the-Fly analysis in seconds. Set 'Analysis_Style' to 'time'
Parameters['Analysis_Delta_Step'] = 1e4      # step interval for On-the-Fly analysis. This will result in non-linear time step for analysis points. Set 'Analysis_Style' to 'step'

Parameters['Deposition_Type'] = 'PLD'            # Type of deposition simulation. Options: 'PLD' or 'MBE'
Parameters['Pulses_per_Layer'] = 20               # number of pulses per monolayer. For PLD only
Parameters['Layers'] = 0.05                         # number of monolayers to simulate

Parameters['Dwell_Time'] = 22.0                   # Total pulse period (for PLD) or total simulation time (for MBE) in seconds.
Parameters['Pulse_Shape'] = 'Uniform'            # Shape of the deposition pulse. Options: 'Uniform', 'Non_Uniform'
Parameters['Pulse_Width'] = 1e-8                 # Width of deposition pulse (for PLD) or total deposition time (for MBE) in seconds. (must be less than dwell time)

Parameters['Post_Anneal'] = False               # Run a Post Anneal Step (True/False)
Parameters['Post_Anneal_Time'] = 100.0          # Length of Post Anneal step in seconds.

In [ ]:
Parameters['Pulses'] = int(Parameters['Pulses_per_Layer']*Parameters['Layers'])
Parameters['Save_File_Name'] = Path.cwd() / 'Test'
Analysis = KMC_Analysis.On_the_Fly_Analysis(Parameters,Path.cwd(),np.zeros([Parameters['Lx'],Parameters['Ly'],Parameters['depth']])) 

In [ ]:
# Plot Analysis Times
print ('Number of Analysis Points per Pulse = ', int(Analysis.analysis_times.shape[0]/(Parameters['Pulses_per_Layer']*Parameters['Layers'])))
plt.figure()
plt.plot(np.arange(Analysis.analysis_times.shape[0]),Analysis.analysis_times,'o')
plt.ylabel('Time (sec)')
plt.xlabel('Points')
# plt.yscale('log')
plt.tight_layout()

# Substrate Generator
* Test out substrate features

In [ ]:
# Substrate Parameters

Parameters['Lx'] = 150           # substrate size in x direction
Parameters['Ly'] = 150           # substrate size in y direction (not used in 2D mode)
Parameters['depth'] = 10         # how many incomplete layers in the lattice (be careful setting this, or errors could occur)

Parameters['Substrate_Type'] = 'Islands'        # Substrate Style . Options: 'Flat', 'Islands', 'Steps', 'Custom'

Parameters['Feature_Layout'] = 'Uniform'        # Feature Layout: 'Uniform': uniform grid, 'Correlated': offset from uniform , 'Random': completely random layout 
Parameters['Feature_Spacing'] = (1,1,0)       # (number in x direction, number in y, avg spread in lattice units)
Parameters['Size_Distribution'] = 'None'        # Size ditribution of features: 'None', 'Gaussian', or 'Correlated'
Parameters['Size_Values'] = (66,1,0)               # (Avg Radius or Step Length, Height of features, Width of distribution)

Parameters['Substrate_Particle_State'] = 'Passive'      # whether substrate particles are active or not. Options: 'Active', 'Passive'.

Parameters['Substrate_File'] = 'Surface_Cov0.0999.npy'  # For custom substrate, name of coordinates file. File must be in same directory as simulation script.
Parameters['Substrate_File'] = 'Surface_Cov0.1999.npy'
Parameters['Substrate_File'] = 'Surface_Cov0.2998.npy'
Parameters['Substrate_File'] = 'Surface_Cov0.3998.npy'
Parameters['Substrate_File'] = 'Surface_Cov0.4997.npy'
# Parameters['Substrate_File'] = 'Surface_Cov0.5997.npy' 
# Parameters['Substrate_File'] = 'Surface_Cov0.6992.npy'
# Parameters['Substrate_File'] = 'Surface_Cov0.7987.npy'
# Parameters['Substrate_File'] = 'Surface_Cov0.8964.npy'
# Parameters['Substrate_File'] = 'Surface_Cov0.9999.npy'

In [ ]:
Substrate = KMC_Misc.Substrate_Generator(Parameters)
List,n = Substrate.Generate_Substrate()
Substrate.Plot()

In [ ]:
# print (List)
print (len(List))
print ('Coverage =',len(List)/(Parameters['Lx']*Parameters['Ly']))

In [ ]:
# calculate step edge density and internal area
surf = Substrate.Surface - 1

# plt.figure()
# plt.imshow(surf)

Lx = surf.shape[0]; Ly = surf.shape[1]
Edge = 0
Area = 0
for i in range(Lx):
    for j in range(Ly):
        if surf[i,j] > 0:
            # count NN
            NN = surf[i-1,j] + surf[(i+1)%Lx,j] + surf[i,j-1] + surf[i,(j+1)%Ly]
            if NN > 0 and NN < 4:
                Edge += 1
            elif NN == 4:
                Area += 1
print (Edge, Area)
print (Area/Edge)
print (Edge+Area)

In [ ]:
Step_Edge = [2119,3102,3758,4172,4260,4065,3633,2986,1962,711]
plt.figure()
plt.plot(Step_Edge)

In [ ]:
L = 50
length = 10
print (int(np.floor(0.5*L/length)))

In [ ]:
for key, value in Parameters.items():
    print (key, value)

In [ ]:
A = [(1,2,3,4),(5,6,7,8),(9,10,11,12)]
B = np.array(A)
C = np.append(B,B,axis=0)
print (C)

In [ ]:
# save a copy of the lattice
SaveData = np.copy(Substrate.Surface)
SaveName = 'Test_Lattice_20x20_R4_Cor'
np.save(SaveName,SaveData)

# Evaluate Deposition Parameters

In [ ]:
# Update test parameters

Parameters['Lx'] = 400            # Substrate size in x direction
Parameters['Ly'] = 400            # Substrate size in y direction (not used in 2D mode)

# Deposition Parameters (Deposition type simulations only)
Parameters['Deposition_Type'] = 'PLD'            # Type of deposition simulation. Options: 'PLD' or 'MBE'
Parameters['Pulses_per_Layer'] = 20               # number of pulses per monolayer. For PLD only
Parameters['Layers'] = 0.05                        # number of monolayers to simulate

Parameters['Dwell_Time'] = 10.0                 # Total pulse period (for PLD) or total simulation time (for MBE) in seconds.
Parameters['Pulse_Shape'] = 'Uniform'            # Shape of the deposition pulse. Options: 'Uniform', 'Non_Uniform'
Parameters['Pulse_Width'] = 1e-8                 # Width of deposition pulse (for PLD) or total deposition time (for MBE) in seconds. (must be less than dwell time)

Parameters['Post_Anneal'] = False               # Run a Post Anneal Step (True/False)
Parameters['Post_Anneal_Time'] = 100.0          # Length of Post Anneal step in seconds.

n = int(Parameters['Lx']*Parameters['Ly']*Parameters['Layers'])
print (n)
Parameters['n'] = n

In [ ]:
Deposition = KMC_Dep.Deposition(Parameters)
Dep_Times, Dep_Coordinates = Deposition.Create_Deposition_Pulse(time=10.0)

In [ ]:
print (Dep_Times.shape)
print (Dep_Coordinates.shape)
print (Dep_Times)
print (Dep_Times[3]-Dep_Times[2])
print (Dep_Coordinates)

In [ ]:
Avg,Peak = KMC_Misc.Deposition(Parameters).Deposition_Rates()
print (Avg,Peak)

# Evaluate Energy Barriers and Rates

In [ ]:
# Energy Barrier Parameters
Parameters['Dimension'] = '3D'  

Parameters['w0'] = 10**13  #1.82*10**13        # Prefactor for Arhenius law (Crystal Vibration Frequency: w0 ~ kT/h )
Parameters['Substrate_Temperature'] = 850     # Substrate temperature in Celcius

Parameters['Ea_diffusion'] = 1.8              # Energy Barrier for surface diffusion in eV
Parameters['Ea_ehrlich_schwoebel'] = 0.0      # Energy Barrier for downhill/uphill diffusion in eV
Parameters['Ea_detach'] = 0.0                 # Energy Barrier for detachment from one nearest neighbor in eV
Parameters['Ea_edge'] = 0.0                   # Energy Barrier for Edge diffusion in eV (3D only). Value of 0 means same as surface diffusion
Parameters['Ea_corner'] = 0.0                 # Energy Barrier for Corner diffusion in eV (3D only)

In [ ]:
Rate_Catalog, rates, moves, keys, num_proc = KMC_Proc.Process_Catalog(Parameters).Return_Rates()

In [ ]:
Rate_Catalog

In [ ]:
print (len(rates))
print (num_proc)

In [ ]:
# Plot Rates as function of Ea


In [ ]:
# Determine energy barrier to get specific Rate
kT = KMC_Misc.Process_Catalog(Parameters).kT
k = 5000  # wanted rate
Ea = -np.log(k/Parameters['w0'])*kT
print (Ea)

In [ ]:
R = 1e6  # D/F ratio

avg_dep_rate = Rate_Catalog['Diff']/R   # avg deposition rate setpoint in monolayers/sec
print ('Avg Dep Rate =',avg_dep_rate)

L = 200    # system size
n = L*L    # number of particles in monolayer
dep_time = 1/avg_dep_rate
print ('Dep Time =',dep_time)

print ('0.2 monolayers',0.2*dep_time)

In [ ]:
# Determine if process can actually happen
Rate = Rate_Catalog['Diff_Detach1_ES']
Max_Particles = 1000*1000

Total_Rate = Rate*Max_Particles
print (Total_Rate)

In [ ]:
# Scaling Law from
D = 91
k = 1.69
r = 3
alpha = D/(r*np.log(1/k))
print (alpha)

# Save Parameters
* create a pickle file to load directly into Input_Script

In [ ]:
# print and check that parameters are set correctly
Parameters